## Introduction
In this lab, you will explore a type of attack called *padding oracle attack*. Let's begin by breaking down what padding oracle attack means word by word.

### Padding Oracle Attack
**Padding**: There are encryption schemes (e.g., a block cipher in the CBC mode) that only support inputs of a fixed length or a multiple of a fixed length. This means it is impossible to use the encryption scheme to encrypt a message $m$ of an arbitrary length directly. Instead, an invertible padding function `pad()` is used so that the padded message $m \mathbin\Vert pad(m)$ has the target fixed length or a multiple of the fixed length.

**Oracle**: In a padding oracle attack. The oracle is typically a remote server that knows the decryption key. The attacker will ask the server to decrypt specially crafted messages, and the server will try to decrypt the messages and tell the attacker if the decryption succeeds or not. For encryption schemes with padding, a part of the decryption process is to check whether the padding is valid. If the server leaks whether the padding is valid to the attacker explicitly, then we say that the server is a *padding oracle*. (This is in contrast to sending the attacker a generic "decryption failure" message regardless of the type of failure that occured during decryption.)

**Attack**: The attacker uses the padding oracle to break the confidentiality of the encryption scheme. Typically, a padding oracle attack aims to recover the plaintext of a given ciphertext.

### Your Task
In this programming assignment, your goal is to implement a padding oracle attack against AES-CBC with a specific padding function. The attack allows an attacker to recover the plaintext of the ciphertext.

### Overview
The assignment is broken down into three components. 
In the first component, your goal is to implement AES-CBC with the said padding function. You are given an implementation of AES and you only need to implement the CBC mode and the padding function.
In the second component, you need to implement the core of the padding oracle attack: an attack that recovers one byte of the plaintext.
In the last component, you will extend the attack in the second component to recover the whole plaintext.

The first component is worth 20 points, the second component is worth 50 points, and the last component is worth 30 points. The whole assignment is worth 100 points. Usual rules regarding acedemic integrity apply.

## Part 1: AES-CBC and the Padding Function (20 points)
### Part 1.1: Padding Function (5 points)
Given a message $m$, we pad the length of $m$ to a multiple of 16 bytes. If $|m|$ is already a multiple of 16, we append a full block. The padding function is defined as follows:

- Let $l$ be the required padding length (in bytes; e.g., if the message is 18-byte long, $l$ should be 14).
- The padding function generates $l$ bytes where each byte equals to $l$ (e.g., a two-byte padding looks like (0x02, 0x02)). 
- The padding is appended to the message.

Implement the padding function as described above.

In [ ]:
def pad(message: bytes) -> bytes:
    block_size = 16
    pad_len = block_size - (len(message) % block_size)

    padding = bytes([pad_len] * pad_len)
    return message + padding

m = b"Example message."
padded_m = pad(m)
print("Original:", m)
print("Padded:", padded_m)


: 

### Part 1.2: AES-CBC (15 points)
Based on the AES implementation below (credit to [bluekeybo](https://github.com/bluekeybo)), implement AES-CBC. The encryption function of AES-CBC should take a message of an arbitrary length as an input. The padding should be done within the encryption function of AES-CBC.
You should also check the validity of the padding in the decryption function. If the validity check fails, you should return an error instead of outputting the decrypted value.

In [ ]:
# This is an implementation of AES in Python
# Supports AES-128, AES-192, AES-256

import numpy as np
import hashlib


class AES:
    def __init__(self, key, key_len=128):
        # AES block size is 16 bytes (128 bits)
        self.block_size = 16
        self.key_len = key_len
        self.key = key

        # AES number of rounds (key_len, rounds): (128, 10), (192,12), (256, 14)
        self.rounds = self.key_len // 32 + 6

        # turn off black formatting
        # fmt: off
        self.S_box = np.array(
            [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
            0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
            0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
            0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
            0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
            0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
            0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
            0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
            0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
            0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
            0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
            0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
            0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
            0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
            0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
            0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16], np.uint8)
        # fmt: on

        # Key expansion
        self.keys = self.KeyExpansion(key=self.key, rounds=self.rounds)

    def KeyExpansion(self, key, rounds):
        # Generating rcon by doubling rcon's previous value in GF(2^8)
        # Only need 10 total rcon values for all AES key lengths (rcon list uses 1-based indexing)
        rcon = [np.zeros(4, dtype="uint8") for _ in range(11)]
        rcon[1][0] = 1
        for i in range(2, 11):
            rcon[i][0] = (rcon[i - 1][0] << 1) ^ (0x11B & -(rcon[i - 1][0] >> 7))

        # N is the length of the key in 32-bit words (i.e. 4-byte words)
        N = self.key_len // 32
        # R is the number of round keys needed: 11 round keys for AES-128, 13 keys for AES-192, and 15 keys for AES-256
        R = rounds + 1

        # Expanded keys for R rounds in 32-bit words (i.e. 4-byte words)
        keys = np.asarray([np.zeros(4, dtype="uint8") for _ in range(4 * R)])

        for i in range(4 * R):
            if i < N:
                keys[i] = key[i]
            elif i % N == 0:
                keys[i] = (
                    keys[i - N] ^ self.S_box[np.roll(keys[i - 1], -1)] ^ rcon[i // N]
                )
            elif (N > 6) and (i % N == 4):
                keys[i] = keys[i - N] ^ self.S_box[keys[i - 1]]
            else:
                keys[i] = keys[i - N] ^ keys[i - 1]

        # Split the keys for each round
        keys = np.split(keys, R)
        # Transpose arrays to match state shape (column-major order) and place in list of keys for each round
        keys = [np.transpose(i) for i in keys]
        return keys

    def AddRoundKey(self, state, key):
        return np.bitwise_xor(state, key)

    def SubBytes(self, state):
        return self.S_box[state]

    def ShiftRows(self, state):
        # Fastest solutions here: https://stackoverflow.com/questions/65177264/fastest-way-to-shift-rows-of-matrix-in-python
        return state.take(
            (0, 1, 2, 3, 5, 6, 7, 4, 10, 11, 8, 9, 15, 12, 13, 14)
        ).reshape(4, 4)

    def MixColumns(self, state):
        # Algorithm for multiplying in Galois Field GF(2^8)
        # https://en.wikipedia.org/wiki/Rijndael_MixColumns
        # From C implementation described in link above modified using the sources below

        def single_col(col):
            # 'col' is a single column of the state
            # 'col_mixed' is the mixed column to be returned
            # 'b' does the multiplication by 2 in GF(2^8)

            # same as in KeyExplansion use of rcon elements
            # reduction modulo the Rijndael polynomial x^8 + x^4 + x^3 + x + 1
            # https://crypto.stackexchange.com/questions/2418/how-to-use-rcon-in-key-expansion-of-128-bit-advanced-encryption-standard

            # Since col is a numpy array, we can perform elementwise operations directly
            b = (col << 1) ^ (0x11B & -(col >> 7))

            # multiplication by 3 is done by b ^ col since 3x=(2 xor 1)x = (2x) xor x
            # https://crypto.stackexchange.com/questions/2402/how-to-solve-mixcolumns/2403#2403
            col_mixed = [
                b[0] ^ col[3] ^ col[2] ^ b[1] ^ col[1],
                b[1] ^ col[0] ^ col[3] ^ b[2] ^ col[2],
                b[2] ^ col[1] ^ col[0] ^ b[3] ^ col[3],
                b[3] ^ col[2] ^ col[1] ^ b[0] ^ col[0],
            ]
            return col_mixed

        state[:, 0] = single_col(state[:, 0])
        state[:, 1] = single_col(state[:, 1])
        state[:, 2] = single_col(state[:, 2])
        state[:, 3] = single_col(state[:, 3])
        return state

    def encrypt(self, plaintext):
        assert len(plaintext) == self.block_size, "Plaintext must be 128 bits."

        # Create the state
        state = (
            np.frombuffer(plaintext, dtype=np.uint8).reshape((4, 4), order="F").copy()
        )

        # AddRoundKey for initial round
        state = self.AddRoundKey(state=state, key=self.keys[0])

        for i in range(1, self.rounds):
            state = self.SubBytes(state=state)
            state = self.ShiftRows(state=state)
            state = self.MixColumns(state=state)
            state = self.AddRoundKey(state=state, key=self.keys[i])

        # Final round (doesn't MixColumns)
        state = self.SubBytes(state=state)
        state = self.ShiftRows(state=state)
        state = self.AddRoundKey(state=state, key=self.keys[self.rounds])

        ciphertext = state.flatten(order="F")

        return ciphertext

### Part 2: Recovering the Last Byte (50 points)
Without loss of generality, let $k$ be the decryption key and $(C_0, C_1)$ be the ciphertext ($C_0$ is the IV). Note that the decryption for the last block of the message (also the first block in this case) is $m_0 = Dec(k, C_1) \oplus C_0$. If the padding length is $l$ (in bytes), then the last $l$ bytes of $m_0$ should be $l$ each.
The unfortunate fact is that the attacker can manipulate $C_0$ and get a corresponding change in $m_0$. There are only two ciphertexts that will be decrypted without error (in the removal of the padding) if the attacker modifies the last byte of $m_0$ (by modifying the last byte of $C_0$), namely $m_0$ itself and $m_0$ except that the last byte is 0x01. This means the attacker can use an exhaustive search (255 attempts at most) to recover the last byte of the message.

Implement an attack that recovers the last byte of the message.

### Part 3: Recovering the Whole Plaintext (30 points)
In fact, the attack in part 2 allows the attacker to remove all of the padding bytes. Extend your attack to recover the actual plaintext.

### Remarks
1. You have seen in the lectures that AES-CBC is a CPA-secure cipher. So why it is still possible for an attacker to break its confidentiality? What capability have we allowed the attacker that is not a part of the CPA security game? Try to formulate a new security game that captures the new capability. (We will see this formally in the lectures later.)
2. The attack you have implemented allows an attacker to recover the plaintext of a ciphertext, but it does not allow you to recover the decryption key. This is not a glitch in the security definition and it is very normal in attacks.
3. You may think that adding integrity protection (by using a MAC) is sufficient to prevent this attack. It turns out that this is not always the case. In particular, SSL 3.0 and TLS 1.0 (roughly speaking, TLS is a rebrand of SSL) pack a message $m$ as $m \mathbin\Vert MAC(K_{MAC}, m) \mathbin\Vert pad(m \mathbin\Vert MAC(K_{MAC}, m))$ before encryption. During decryption, the validity of the padding is checked *before* the validity of the MAC, and the "invalid padding" message is sent to the client explicitly. So nothing is stopping the attack you have implemented from working! For more padding oracle attacks on the CBC mode, see the following paper: [Serge Vaudenay. Security Flaws Induced by CBC Padding Applications to SSL, IPSEC, WTLS...](https://www.iacr.org/archive/eurocrypt2002/23320530/cbc02_e02d.pdf).
4. The reason why the key is referred to as the decryption key in this assignment is that there are also padding oracle attacks on public-key encryption schemes (where the encryption key and decryption key are different). You will see an example of a padding oracle attack on a public-key encryption scheme in a later assignment.